In [ ]:
# https://twitter.com/p_surridge/status/925046405617594368

# antiIntellectual
# I’d rather put my trust in the wisdom of ordinary people than the opinions of experts
# W7W8W9W10W11

# education

# Any breakdown of these people by party vote or media consumption.
# It says alot by itself, but a semi unreality to it an actual practice!


In [1]:
import numpy as np
import pandas as pd
np.random.seed(0)

import matplotlib.pyplot as plt
import gc
import re

from sklearn import datasets
from sklearn.decomposition import PCA, IncrementalPCA, NMF, TruncatedSVD, FastICA, FactorAnalysis, SparsePCA
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from itertools import cycle
from IPython.display import display
import pickle, os

import seaborn as sns

BES_data_folder = "../BES_analysis_data/"
BES_code_folder = "../BES_analysis_code/"

# from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE
from gaussian_kde import gaussian_kde
from utility import display_components,display_pca_data


import time
import warnings

import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

%matplotlib inline

In [2]:
from sklearn import datasets, cluster
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn import cluster

In [3]:
Treatment = "W13_only" 
if not os.path.exists(BES_data_folder+Treatment):
    os.makedirs(BES_data_folder+Treatment)

In [4]:
dataset = "BES2017_W13_v1.0.dta"
wave = "W13_only"

BES_Panel = pd.read_stata(BES_data_folder+dataset)

In [10]:
BES_Panel["edlevel"].value_counts()

Undergraduate        8526
GCSE A*-C            6120
A-level              5822
Postgrad             2640
No qualifications    2078
GCSE D-G             1315
Name: edlevel, dtype: int64

In [11]:
BES_Panel["edlevel"].value_counts().sum()

26501

In [8]:
BES_Panel.shape

(31197, 312)

In [16]:
BES_Panel["education"].value_counts()

University or CNAA first degree (eg BA, B.Sc, B.Ed)      6199
GCE A level or Higher Certificate                        4245
CSE grade 1, GCE O level, GCSE, School Certificate       4146
Other technical, professional or higher qualification    3793
University or CNAA higher degree (eg M.Sc, Ph.D)         2640
No formal qualifications                                 2078
City and Guild certificate                               1342
University diploma                                       1158
Teaching qualification (not degree)                       703
City and Guild certificate - advanced                     688
CSE grades 2-5                                            673
Scottish Higher Certificate                               596
Clerical and commercial                                   573
Prefer not to say                                         571
Nursing qualification (eg SEN, SRN, SCM, RGN)             466
Recognised trade apprenticeship completed                 422
Don't kn